In [1]:
# Set working directory (optional during development)
import os
os.chdir('/Users/sudishmakarki/My_project2')  # only if needed
print(" Working directory:", os.getcwd())

# Standard Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Custom Functions
from models.data_interpolation import (
    load_data,
    preprocess_data,
    split_train_test,
    generate_time_series_splits
)

from models.model_prophet import (
    prepare_prophet_data,
    train_baseline_prophet,
    forecast_with_model,
    calculate_peak_hours,
    evaluate_metrics,
    cross_validate_baseline
)

sns.set(style="whitegrid")

 Working directory: /Users/sudishmakarki/My_project2


In [2]:
# Load and preprocess
df = load_data('data/RestaurantData.csv')
df_clean = preprocess_data(df)

                     Year  Month  Season  DayOfWeek WeekDay  Hour  \
Timestamp                                                           
2018-01-01 00:00:00  2018      1  Winter          1  Monday     0   
2018-01-01 01:00:00  2018      1  Winter          1  Monday     1   
2018-01-01 02:00:00  2018      1  Winter          1  Monday     2   
2018-01-01 03:00:00  2018      1  Winter          1  Monday     3   
2018-01-01 04:00:00  2018      1  Winter          1  Monday     4   

                            Holiday Weather SpecialEvent  CustomerCount  \
Timestamp                                                                 
2018-01-01 00:00:00  New Year's Day   Rainy          NaN              6   
2018-01-01 01:00:00  New Year's Day   Windy          NaN             11   
2018-01-01 02:00:00  New Year's Day   Snowy          NaN              9   
2018-01-01 03:00:00  New Year's Day   Rainy          NaN             10   
2018-01-01 04:00:00  New Year's Day  Cloudy          NaN          

In [3]:
# Split into train and test
restaurant_train, restaurant_test = split_train_test(df_clean, split_date='2022-01-01')
# Format for Prophet
restaurant_train_prophet, restaurant_test_prophet = prepare_prophet_data(restaurant_train, restaurant_test)
# Train the baseline Prophet model
m = train_baseline_prophet(restaurant_train_prophet)

In [4]:
# -- Test Set Forecasting --
# Predict on the test set and display the first few rows
restaurant_test_fcst = forecast_with_model(m, restaurant_test_prophet)
print("Forecast on Test Set (first 5 rows):")
print(restaurant_test_fcst.head())


Forecast on Test Set (first 5 rows):
                   ds      trend  yhat_lower  yhat_upper  trend_lower  \
0 2022-01-01 00:00:00  20.446485   -0.441093   18.161786    20.446485   
1 2022-01-01 01:00:00  20.446561    1.339577   19.947939    20.446561   
2 2022-01-01 02:00:00  20.446637    1.993763   21.653017    20.446637   
3 2022-01-01 03:00:00  20.446713   -0.338351   18.945746    20.446713   
4 2022-01-01 04:00:00  20.446789   -0.932230   18.649674    20.446789   

   trend_upper  additive_terms  additive_terms_lower  additive_terms_upper  \
0    20.446485      -11.624321            -11.624321            -11.624321   
1    20.446561       -9.794884             -9.794884             -9.794884   
2    20.446637       -8.705095             -8.705095             -8.705095   
3    20.446713      -10.132657            -10.132657            -10.132657   
4    20.446789      -11.764320            -11.764320            -11.764320   

       daily  daily_lower  daily_upper    weekly  weekl

In [5]:
# Calculate average forecast per hour
hourly_avg = restaurant_test_fcst.groupby('Hour')['yhat'].mean()
print("\nAverage Forecast by Hour:")
display(hourly_avg)


Average Forecast by Hour:


Hour
0      7.246245
1      9.169158
2     10.371492
3      9.074454
4      7.590038
5      8.459568
6     11.104544
7     12.760900
8     12.392444
9     12.769541
10    17.787457
11    27.462583
12    36.149942
13    36.782499
14    27.776974
15    15.894710
16    11.539598
17    20.027610
18    36.537792
19    49.516678
20    49.785918
21    37.493399
22    21.121063
23     9.987529
Name: yhat, dtype: float64